# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

## 2. Install python SDK for bedrock

In [ ]:
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip
!unzip bedrock-python-sdk.zip -d bedrock-sdk
!rm -rf bedrock-python-sdk.zip

In [ ]:
install_needed = True

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-*-py3-none-any.whl
    !{sys.executable} -m pip install langchain==0.0.286
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U urllib
    !{sys.executable} -m pip install -U librosa
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

## 3. Check setting
아래 메시지가 뜨면 정상 <BR>
![nn](../imgs/check.png)


In [ ]:
import boto3
import langchain

In [ ]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

In [ ]:
is_internal_use = <internal-use> ## if 고객: False, aws직원: True

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [ ]:
bedrock_region = <your region> ## "us-east-1" or "us-west-2"

In [ ]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [ ]:
bedrock_config

In [ ]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [ ]:
output_text = bedrock_client.list_foundation_models()
output_text 

## 4. generation

In [ ]:
from termcolor import colored
from langchain.llms.bedrock import Bedrock

In [ ]:
llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [ ]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [ ]:
print (colored(llm(prompt), "green"))